In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from SktWsegRWR_utf8 import *
import pickle
import ProbData
from ProbModels import *
import multiprocessing
import math
import json
import pprint
import csv
from utilities import *
from IPython.display import display
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import pandas as pd
%matplotlib inline

Dataloader Started[Prob]...
Dataloader Finished[Prob]...


In [4]:
data1 = json.load(open('../NewData/data1.json', 'r'))
data2 = json.load(open('../NewData/data2.json', 'r'))

In [7]:
len(data1)

66817

In [14]:
def func():
    absent = []
    oldKeys = [rom_slp(k) for k in ProbData.fullCo_oc_mat.keys()]
    for key in data1.keys():
        if key not in oldKeys:
            absent.append(key)

In [16]:
absent

['sima']

In [18]:
len(set(oldKeys))

81910